In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import random

import cv2, os, shutil
from matplotlib import pyplot as plt

In [ ]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White

def visualize_bbox(img, bbox, class_name, pic_w, pic_h, color=BOX_COLOR, thickness=2):
    dataType = "yolo"

    """Visualizes a single bounding box on the image"""

    if dataType == 'coco':
        x_min, y_min, w, h = bbox # 정규화 된 0~1 사이의 값
        x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
    elif dataType == "yolo":
        x_center, y_center, w, h = bbox
        # 픽셀 좌표로 변환 이미지의 width와 height값
        x_min = int(float(x_center - w/2) * pic_w)
        x_max = int(float(x_center + w/2) * pic_w)
        y_min = int(float(y_center - h/2) * pic_h)
        y_max = int(float(y_center + h/2) * pic_h)
    print(w, h)
    print(x_min, y_min, y_min, y_max)
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)

    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSH\nEY_SIMPLEX,
        fontScale=0.35,
        color=TEXT_COLOR,
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name, img_shape):
    img = image.copy()
    h,w = img_shape[0:2]
    print(bboxes)
    print(category_ids)
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name, w, h)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [ ]:
def read_label_txt(txtFile):
    category_ids = []
    bboxes = []

    f=open(txtFile,'r')

    while True:
        line = f.readline()
        if not line: break
        ids, xc, yc, w, h= line.split(' ')
        category_ids.append(int(ids))
        bboxes.append([float(xc),float(yc),float(w),float(h)])
        #print(line)
    f.close()
    return category_ids, bboxes

def write_label_txt(txtFile, category_ids, bboxes):
    f=open(txtFile,'w')

    for i, ids in enumerate(category_ids):
        xc,yc,w,h = bboxes[i]
        f.write("{} {} {} {} {}\n".format(int(ids),xc,yc,w,h))
        #print("{} {}".format(int(ids), bboxes[i]))
    f.close()

In [41]:
trainRatio = 0.7
validRatio = 0.15
testRatio = 1 - trainRatio - validRatio

In [74]:
import os, shutil

# 데이터셋 디렉토리 경로 설정 (데이터셋이 저장된 경로로 수정하세요)
dataset_dir = os.path.join(os.getcwd(), 'sepImgLabSimpson')

# 새로운 train, valid, test 디렉토리 생성 경로
base_dir = os.path.join(os.getcwd(), 'sepImgLabSimpson_tvt')

# 클래스 목록
classes = ['bart', 'homer', 'lisa', 'maggie'] #marge
fileTypes = ['images','labels']

# 폴더 경로 생성
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

# 폴더 생성 함수
def create_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

def checkEqual(filesPath, label):
    img_mismatch = True
    label_mismatch = True
    img_mismatch_list = []
    label_mismatch_list = []
    
    images_path = os.path.join(filesPath, str(label + '_images'))
    labels_path = os.path.join(filesPath, str(label + '_labels'))
    imageFiles = os.listdir(images_path)
    labelFiles = os.listdir(labels_path)
    if len(imageFiles) != len(labelFiles):
        print(f'{label} imageFiles and labelFiles not equal')
        for image in imageFiles:
            for label in labelFiles:
                if os.path.splitext(image)[0] == os.path.splitext(label)[0]:
                    img_mismatch = False
                    break
            if img_mismatch:
                img_mismatch_list.append(image)
            img_mismatch = True
        for label in labelFiles:
            for image in imageFiles:
                if os.path.splitext(label)[0] == os.path.splitext(image)[0]:
                    label_mismatch = False
                    break
            if label_mismatch:
                label_mismatch_list.append(label)
            label_mismatch = True
                
        if img_mismatch_list:
            print(f'no label for the following images: {img_mismatch_list}')
        if label_mismatch_list:    
            print(f'no image for the following labels: {label_mismatch_list}')
        return False
    return True 

# train, valid, test 폴더 및 각각의 클래스 폴더 생성
# 각각의 클래스 폴더 및 이미지와 레이블 폴더 생성 
for cls in classes:
    create_dir(os.path.join(train_dir, cls))
    create_dir(os.path.join(valid_dir, cls))
    create_dir(os.path.join(test_dir, cls))
    for fileType in fileTypes:
        create_dir(os.path.join(train_dir, cls, fileType))
        create_dir(os.path.join(valid_dir, cls, fileType))
        create_dir(os.path.join(test_dir, cls, fileType))

# 이미지 복사 함수
def copy_images_and_label(start_idx, end_idx, src_dir, dst_dir, label):
    src_img_dir = os.path.join(src_dir, str(label + '_images'))
    src_label_dir = os.path.join(src_dir, str(label + '_labels'))
    srcImages = os.listdir( src_img_dir )
    srcLabels = os.listdir( src_label_dir )
    for i in range(start_idx, end_idx + 1):
        image_name = f'{label}.{i}.jpg'
        label_name = f'{label}.{i}.txt'
        src_imagePath = os.path.join(src_img_dir, srcImages[i])
        src_labelPath = os.path.join(src_label_dir, srcLabels[i])
        dst_imagePath = os.path.join(dst_dir, 'images', image_name)
        dst_labelPath = os.path.join(dst_dir, 'labels', label_name)
        if os.path.exists(src_imagePath):
            shutil.copy(src_imagePath, dst_imagePath)
        if os.path.exists(src_labelPath):
            shutil.copy(src_labelPath, dst_labelPath)

# 클래스 별로 train, valid, test 데이터셋 구성
for cls in classes:
    #retrieve image count
    filesPath = os.path.join(os.getcwd(), 'sepImgLabSimpson', cls)

    if checkEqual(filesPath, cls):
    
        files = os.listdir( os.path.join(filesPath, str(cls + '_images')))
        
        train_num = int(trainRatio * len(files))
        valid_num = int(validRatio * len(files))
        test_num  = len(files) - train_num - valid_num 
    
        train_range = [0, train_num-1]
        valid_range = [train_num, train_num + valid_num -1]
        test_range  = [train_num + valid_num, train_num + valid_num + test_num-1]
        print(f'train {train_range} valid {valid_range} test {test_range}')
    
        #copy images and label
        copy_images_and_label(train_range[0], train_range[1], os.path.join(dataset_dir, cls), os.path.join(train_dir, cls), cls)
    
        # valid dataset 구성 (1000~1249)
        copy_images_and_label(valid_range[0], valid_range[1], os.path.join(dataset_dir,cls), os.path.join(valid_dir, cls), cls)
    
        # test dataset 구성 (1250~1499)
        copy_images_and_label(test_range[0], test_range[1], os.path.join(dataset_dir,cls), os.path.join(test_dir, cls), cls)

print("데이터셋 분할 및 복사가 완료되었습니다.")

train [0, 938] valid [939, 1139] test [1140, 1341]
train [0, 1129] valid [1130, 1371] test [1372, 1614]
train [0, 946] valid [947, 1148] test [1149, 1352]
train [0, 88] valid [89, 107] test [108, 127]
데이터셋 분할 및 복사가 완료되었습니다.


In [75]:
from ultralytics import YOLO
model = YOLO('yolo11n.pt')

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
results = model.train(data='data.yaml', epochs=100, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.13 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train29, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning C:\Users\SBA\yolo\yolov11\kDataset_yolo\train\labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%
C:\Users\sba\miniconda3\envs\album\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


C:\Users\sba\miniconda3\envs\album\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning C:\Users\SBA\yolo\yolov11\kDataset_yolo\valid\labels.cache... 7 images, 0 backgrounds, 0 corrupt: 100%|██


Plotting labels to runs\detect\train29\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train29
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      1.183      2.672      1.332         34        640: 100%|██████████| 2/2 [00:20<00:00, 10.50
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<0

                   all          7         14    0.00667          1      0.569      0.214



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/2 [00:00<?, ?it/s]


KeyboardInterrupt: 